# Stock Predictions

## Imports

In [8]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

C:\Users\cjhun\anaconda3\envs\dojo-env\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
spy = yf.download('SPY')

[*********************100%***********************]  1 of 1 completed


In [4]:
spy

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.968750,43.968750,43.750000,43.937500,24.941389,1003200
1993-02-01,43.968750,44.250000,43.968750,44.250000,25.118778,480500
1993-02-02,44.218750,44.375000,44.125000,44.343750,25.172007,201300
1993-02-03,44.406250,44.843750,44.375000,44.812500,25.438097,529400
1993-02-04,44.968750,45.093750,44.468750,45.000000,25.544540,531500
...,...,...,...,...,...,...
2023-11-30,455.480011,456.760010,453.339996,456.399994,456.399994,79752700
2023-12-01,455.769989,459.649994,455.160004,459.100006,459.100006,89097900
2023-12-04,455.600006,459.119995,454.339996,456.690002,456.690002,72430900


In [5]:
spy.index = pd.to_datetime(spy.index)

# Split the data into training and testing sets
train_data = spy.iloc[:int(0.8 * len(spy)), :]
test_data = spy.iloc[int(0.8 * len(spy)):, :]

In [9]:
features = ['Open', 'Volume']
target = 'Close'

# Create and train model
model = xgb.XGBRegressor()
model.fit(train_data[features], train_data[target])

C:\Users\cjhun\anaconda3\envs\dojo-env\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [12]:
# Make and show the predictions on the test data
predictions = model.predict(test_data[features])
print('Model Predictions:')
print(predictions)

Model Predictions:
[252.32759 252.31757 252.37535 ... 252.32114 252.4364  252.31499]
